In [1]:
import pandas as pd
import geopandas as gpd

In [20]:
# don't update these that have been locked
lstLockForecastArea = ['WFRC']

# GLOBAL VARIABLES

dfObsVolSource = pd.DataFrame([
    ['AADTHistory.xlsx'                  , [0,2,3,4,5]],
    ['WF_Segments_20240326_Draft.shp'    , [1]        ]
],columns=('SOURCE','SUBAREAID'))

dfObsVolSource = dfObsVolSource.explode('SUBAREAID')

dfObsVolSource.to_csv('intermediate/obsvolsource.csv',index=False)

# base year adjustment
# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
baseYear = 2019

dfFactorSource = pd.DataFrame([
    ['Segments_State_20231221_Draft.shp' , [0,2,3,4,5]],
    ['WF_Segments_20240326_Draft.shp'    , [1]        ]
],columns=('SOURCE','SUBAREAID'))

dfFactorSource = dfFactorSource.explode('SUBAREAID')

# factors file from last notebook
fnFactors = 'intermediate/factors.csv'

# field for weekday factor
colFacWdAvg = 'FAC_WDAVG'
colFacSpr   = 'FAC_SPR'
colFacFal   = 'FAC_FAL'
# SHOULD WE ADJUST WEEKDAY FACTOR INTO FUTURE?? 

# some models are AWDT for whole year and some are AADT for a season
# this list of SUBAREAIDs is for models that are AADT for season and need to be adjusted by a Spring/Fall factor
subareaidsThatUseSprFalFac = [4]

# where the same segid in multiple subareas and model, drop order for duplicate segids... pick order to drop from models
duplicateSegsInTdms = pd.DataFrame([
    ['0015_042.2', 3],
    ['0065_002.4', 0],
    ['0150_029.4', 0],
    ['0150_030.0', 0],
    ['0150_033.3', 0],
    ['1822_000.0', 0],
    ['2863_000.0', 4],
    ['2865_019.4', 4]
], columns=('SEGID','removeSUBAREAID'))

duplicateSegsInTdms.to_csv('intermediate/duplicateSegsInTdms.csv',index=False)

# where the same segid in multiple subareas and model, drop order for duplicate segids... pick order to drop from models
hardCodeSegIds = pd.DataFrame([
    ['0006_222.1', 0]
], columns=('SEGID','hardCodeSUBAREAID'))

hardCodeSegIds.to_csv('intermediate/hardCodeSegIds.csv',index=False)

dfModelSegSummaries = pd.DataFrame( [
    [0, 2019, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_BY2019.csv'     ],
    [0, 2023, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_OY2023.csv'     ],
    [0, 2028, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_STIP2028.csv'   ],
    [0, 2032, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2032.csv'    ],
    [0, 2042, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2042.csv'    ],
    [0, 2050, 'data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2050.csv'    ],
    [1, 2019, 'data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv'                         ],
    [1, 2023, 'data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv'                         ],
    [1, 2028, 'data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv'                     ],
    [1, 2032, 'data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv'                     ],
    [1, 2042, 'data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv'                     ],
    [1, 2050, 'data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv'                     ],
    [2, 2019, 'data/model-output/2_Cache/BY2019_Summary_SEGID.dbf'                             ],
    [2, 2023, 'data/model-output/2_Cache/BY2023_Summary_SEGID.dbf'                             ],
    [2, 2028, 'data/model-output/2_Cache/Summary_SEGID_STIP28.dbf'                             ],
    [2, 2032, 'data/model-output/2_Cache/Summary_SEGID_LRP23_32.dbf'                           ],
    [2, 2042, 'data/model-output/2_Cache/Summary_SEGID_LRP23_42.dbf'                           ],
    [2, 2050, 'data/model-output/2_Cache/Summary_SEGID_LRP23_50.dbf'                           ],
    [3, 2019, 'data/model-output/3_Dixie/Summary_SEGID_19.dbf'                                 ],
    [3, 2023, 'data/model-output/3_Dixie/Summary_SEGID_23.dbf'                                 ],
    [3, 2028, 'data/model-output/3_Dixie/Summary_SEGID_28.dbf'                                 ],
    [3, 2032, 'data/model-output/3_Dixie/Summary_SEGID_32.dbf'                                 ],
    [3, 2042, 'data/model-output/3_Dixie/Summary_SEGID_42.dbf'                                 ],
    [3, 2050, 'data/model-output/3_Dixie/Summary_SEGID_50.dbf'                                 ],
    [4, 2019, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_BY2019.dbf'           ],
    [4, 2023, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_OY2023.dbf'           ],
    [4, 2028, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_STIP2028.dbf'         ],
    [4, 2032, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2032.dbf'          ],
    [4, 2042, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2042.dbf'          ],
    [4, 2050, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2050.dbf'          ],
    [5, 2019, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_BY2019.dbf'  ],
    [5, 2023, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_OY2023.dbf'  ],
    [5, 2028, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_STIP2028.dbf'],
    [5, 2032, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2032.dbf'  ],
    [5, 2042, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2042.dbf'  ],
    [5, 2050, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2050.dbf'  ]
], columns=('modSubareaId','modYear','modSegSummaryFile'))

# seg summary fields to be useds
use_fields = ['SEGID','SUBAREAID','DY_VOL','FT','ATYPENAME','LANES']

# drop the following records that have following SEGIDs
dropSegId = ['NO','N0']

#display(dfModelSegSummaries)


In [24]:
# Required libraries
from dbfread import DBF
import pandas as pd

# Create a list to store DataFrames read from each file
frames = []

# Iterate through the rows and read each file
for index, row in dfModelSegSummaries.iterrows():
    # Prepend path with 'dbfs:/'
    file_path = row['modSegSummaryFile']
    print(file_path)
    
    # Check the file extension
    if file_path.endswith('.dbf'):
        # Read the .dbf file with dbfread
        table = DBF(file_path)
        df = pd.DataFrame(iter(table))  # Convert the DBF table to a DataFrame
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path, low_memory=False)
    else:
        # If the file is neither .csv nor .dbf, skip this iteration
        print(f"Unsupported file format for file: {file_path}")
        continue

    df = df[use_fields]

    # Update SUBAREAID in df where there's a matching SEGID
    mapping = hardCodeSegIds.set_index('SEGID')['hardCodeSUBAREAID']
    df['SUBAREAID'] = df['SEGID'].map(mapping).fillna(df['SUBAREAID'])

    # filter out where link SUBAREAID match the model SUBAREAID... Mostly for USTM that spans multiple model spaces
    df = df[df['SUBAREAID']==row['modSubareaId']]

    # drop SEGIDs defined in dropSegId list
    df = df[~df['SEGID'].isin(dropSegId)]

    df['YEAR'] = row['modYear'] # Add modYear column

    frames.append(df)

# Concatenate all the frames into a single DataFrame
dfMdlVol = pd.concat(frames, ignore_index=True)

# If you're using Databricks, use 'display' to show the DataFrame
display(dfMdlVol)

data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_BY2019.csv
data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_OY2023.csv
data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_STIP2028.csv
data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2032.csv
data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2042.csv
data/model-output/0_USTM_v3.0 - 2024-02-20_DRAFT/_Summary_SEGID_LRP2050.csv
data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv
data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv
data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv
data/model-output/2_Cache/BY2019_Summary_SEGID.dbf
data/model-output/2_Cache/BY2023_Summary_SEGID.dbf
data/model-output/2_Cache/Summary_SEGID_STIP28.dbf
data/model-output/2_Cache/Summary_SEGID_LRP23_

,SEGID,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR
0,0006_000.0,0.0,126.5,2.0,Rural,1.0,2019
1,0006_000.7,0.0,213.6,2.0,Rural,1.0,2019
2,0006_016.0,0.0,76.9,2.0,Rural,1.0,2019
3,0006_046.0,0.0,76.9,2.0,Rural,1.0,2019
4,0006_060.2,0.0,80.3,2.0,Rural,1.0,2019
...,...,...,...,...,...,...,...
49500,3218_006.4,5.0,49.5,7.0,Rural,2.0,2050
49501,UDOT_7006,5.0,4181.0,4.0,Transition,4.0,2050
49502,UDOT_7007,5.0,7259.0,4.0,Rural,4.0,2050
49503,UDOT_7008,5.0,453.9,4.0,Rural,2.0,2050


# Add Factors

In [34]:
# read intermediate CSV with base year observed AADT


In [182]:
dfObsVolSource

,SOURCE,SUBAREAID
0,AADTHistory.xlsx,0
0,AADTHistory.xlsx,2
0,AADTHistory.xlsx,3
0,AADTHistory.xlsx,4
0,AADTHistory.xlsx,5
1,WF_Segments_20240326_Draft.shp,1


In [188]:
gdfSegments = pd.read_csv('intermediate/segments.gdf',low_memory=False)

# use cofips to match to segments that don't have factors

dfSegsForFacSource = pd.merge(gdfSegments[['SEGID','CO_FIPS','SUBAREAID','SOURCE']],dfFactorSource, on=('SUBAREAID','SOURCE'))

dfSegsForFacSource.fillna(0,inplace=True)

dfSegsForFacSource = dfSegsForFacSource[~dfSegsForFacSource['SEGID'].str.contains('UTA_')]

dfSegsForFacSource.groupby(['SEGID']).agg(COUNT=('SOURCE','count'))

,COUNT
SEGID,
0006_000.0,1
0006_000.7,1
0006_016.0,1
0006_046.0,1
0006_060.2,1
...,...
WFRC_8472,1
WFRC_8473,1
WFRC_8474,1


In [189]:
# Correcting the syntax for renaming columns and merging
dfMdlVolWithAllSegs = pd.merge(
    dfSegsForFacSource.rename(columns={'SUBAREAID': 'SAID_FAC'}), 
    dfMdlVol, 
    on='SEGID',
    how='left'
)

# FILL SEGS THAT DON'T HAVE MODEL WITH BASIC ASSUMPTIONS
dfMdlVolWithAllSegs['YEAR'].fillna(2019, inplace=True)
dfMdlVolWithAllSegs.fillna(0, inplace=True)
dfMdlVolWithAllSegs

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,103.6,2.0,Rural,1.0,2032.0
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,107.9,2.0,Rural,1.0,2042.0
...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0


In [190]:
dfMdlVolWithAllSegs[dfMdlVolWithAllSegs['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR
9448,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0


In [191]:
# GET WEEKDAY FACTORS

# read in factors
_dfFac = pd.read_csv(fnFactors, usecols=('SOURCE', 'SEGID', colFacWdAvg, colFacSpr, colFacFal))

_dfFac['FAC_SPRFAL'] = (_dfFac[colFacSpr] + _dfFac[colFacFal]) / 2

dfMdlVolWithFac = pd.merge(dfMdlVolWithAllSegs, _dfFac, on=('SEGID','SOURCE'), how="left")

dfMdlVolWithFac.fillna(0, inplace=True)

display(dfMdlVolWithFac)

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.0316,1.0276,0.9840,1.02960
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.0316,1.0276,0.9840,1.02960
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,103.6,2.0,Rural,1.0,2032.0,1.0316,1.0276,0.9840,1.02960
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,107.9,2.0,Rural,1.0,2042.0,1.0316,1.0276,0.9840,1.02960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.0243,1.0104,1.0924,1.01735
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.0243,1.0104,1.0924,1.01735
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.0243,1.0104,1.0924,1.01735
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.0243,1.0104,1.0924,1.01735


In [192]:
dfMdlVolWithFac[dfMdlVolWithFac['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
9448,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0827,1.2456,0.9243,1.16415


## get and use avg where factors zero... no match

In [193]:
dfFindZeroFac = dfMdlVolWithFac.copy()
dfFindZeroFac.loc[(dfFindZeroFac['FAC_WDAVG']==0), 'checkZero'] = 1
dfFindZeroFac.groupby(['CO_FIPS','YEAR']).agg(numSegs=('SEGID','count'),numSegsWithFacZero=('checkZero','sum'))

numSegs  numSegsWithFacZero
CO_FIPS YEAR                               
1.0     2019.0       96                 0.0
        2023.0       80                 0.0
        2028.0       80                 0.0
        2032.0       80                 0.0
        2042.0       80                 0.0
...                 ...                 ...
57.0    2023.0      548                 0.0
        2028.0      548                 0.0
        2032.0      554                 0.0
        2042.0      556                 0.0
        2050.0      568                 0.0

[174 rows x 2 columns]

In [194]:
dfNonZeroFac = dfFindZeroFac[~(dfFindZeroFac['checkZero']== 1)].drop(columns=['checkZero'])
display(dfNonZeroFac)
dfZeroFac    = dfFindZeroFac[ (dfFindZeroFac['checkZero']== 1)].drop(columns=['checkZero'])
display(dfZeroFac)

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.0316,1.0276,0.9840,1.02960
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.0316,1.0276,0.9840,1.02960
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,103.6,2.0,Rural,1.0,2032.0,1.0316,1.0276,0.9840,1.02960
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,107.9,2.0,Rural,1.0,2042.0,1.0316,1.0276,0.9840,1.02960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.0243,1.0104,1.0924,1.01735
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.0243,1.0104,1.0924,1.01735
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.0243,1.0104,1.0924,1.01735
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.0243,1.0104,1.0924,1.01735


,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
50303,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.0,0.0,0.0,0.0
50304,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.0,0.0,0.0,0.0


In [195]:
dfZeroFacSaAtCombos = dfZeroFac[['CO_FIPS','ATYPENAME']].drop_duplicates()
display(dfZeroFacSaAtCombos)

,CO_FIPS,ATYPENAME
50303,35.0,0


In [196]:
dfMdlVolWithFac_Filtered = pd.merge(dfMdlVolWithFac, dfZeroFacSaAtCombos, on=['CO_FIPS', 'ATYPENAME'], how='inner')
dfMdlVolWithFac_Filtered


,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0190_015.7,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.8387,0.7791,0.7907,0.80890
1,0190_016.1,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.8387,0.7791,0.7907,0.80890
2,0209_000.4,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0316,1.0276,0.9840,1.02960
3,0210_012.3,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.7781,0.9888,0.8297,0.88345
4,2031_000.0,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0324,1.0107,1.0918,1.02155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,WFRC_8469,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0243,1.0104,1.0924,1.01735
147,WFRC_8470,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0324,1.0107,1.0918,1.02155
148,WFRC_8472,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0095,1.0117,1.0946,1.01060
149,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,0.0000,0.0000,0.0000,0.00000


In [197]:
dfMdlVolWithFac_Filtered_avg = dfMdlVolWithFac_Filtered.groupby(['CO_FIPS','ATYPENAME'],as_index=False).agg(FAC_FAL = ('FAC_FAL','mean'),FAC_SPR = ('FAC_SPR','mean'),FAC_WDAVG = ('FAC_WDAVG','mean'),FAC_SPRFAL = ('FAC_SPRFAL','mean'))
dfMdlVolWithFac_Filtered_avg

,CO_FIPS,ATYPENAME,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,35.0,0,1.007782,0.996462,1.058044,1.002122


In [198]:
dfZeroFacWithAvg = pd.merge(dfZeroFac.drop(columns=['FAC_FAL','FAC_SPR','FAC_WDAVG','FAC_SPRFAL']),dfMdlVolWithFac_Filtered_avg, on=('CO_FIPS','ATYPENAME'),how='left')
dfZeroFacWithAvg

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122
1,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122


In [199]:
dfMdlVolWithFac = pd.concat([dfZeroFacWithAvg,dfNonZeroFac], ignore_index=True)
dfMdlVolWithFac

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122
1,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.031600,1.027600,0.984000,1.029600
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.031600,1.027600,0.984000,1.029600
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.031600,1.027600,0.984000,1.029600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.024300,1.010400,1.092400,1.017350
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.024300,1.010400,1.092400,1.017350
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.024300,1.010400,1.092400,1.017350
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.024300,1.010400,1.092400,1.017350


# drop where overlapping forecasts (same segids in different subareas)

In [200]:
# get segids only
dfSegIds = dfMdlVolWithFac[['SEGID']].drop_duplicates()
display(dfSegIds)

# filter by 
display(dfMdlVolWithFac[['SEGID','SUBAREAID']].drop_duplicates())

,SEGID
0,WFRC_8474
1,WFRC_8475
2,0006_000.0
8,0006_000.7
14,0006_016.0
...,...
50294,WFRC_8470
50295,WFRC_8471
50298,WFRC_8472
50299,WFRC_8473


,SEGID,SUBAREAID
0,WFRC_8474,0.0
1,WFRC_8475,0.0
2,0006_000.0,0.0
8,0006_000.7,0.0
14,0006_016.0,0.0
...,...,...
50294,WFRC_8470,0.0
50295,WFRC_8471,1.0
50298,WFRC_8472,0.0
50299,WFRC_8473,1.0


In [201]:
# Assuming dfMdlVolWithFac is your initial DataFrame
df_filtered = dfMdlVolWithFac[['SEGID','SUBAREAID','SAID_FAC']].drop_duplicates()

df_filtered.rename(columns={'SUBAREAID':'TDM_SUBAREAID','SAID_FAC':'SEG_SUBAREAID'}, inplace=True)

# Find duplicates based on 'SEGID'
duplicates = df_filtered.duplicated(subset=['SEGID'], keep=False)

# Filter the DataFrame to include only the duplicates
df_duplicates = df_filtered[duplicates]

df_duplicates = df_duplicates.sort_values(by=['SEGID','TDM_SUBAREAID'])

# This will give you a DataFrame with only the rows where 'SEGID' is duplicated
df_duplicates

,SEGID,TDM_SUBAREAID,SEG_SUBAREAID
16638,0015_042.2,3.0,5.0
16644,0015_042.2,5.0,5.0
24286,0065_002.4,0.0,1.0
24292,0065_002.4,1.0,1.0
5962,0150_029.4,0.0,0.0
5968,0150_029.4,4.0,0.0
5974,0150_030.0,0.0,0.0
5980,0150_030.0,4.0,0.0
5986,0150_033.3,0.0,0.0
5992,0150_033.3,4.0,0.0


In [202]:
# Assuming dfMdlVolWithFac is your initial DataFrame
df_filtered = dfMdlVolWithFac[['SEGID','SUBAREAID','SAID_FAC']].drop_duplicates()

df_filtered.rename(columns={'SUBAREAID':'TDM_SUBAREAID','SAID_FAC':'SEG_SUBAREAID'}, inplace=True)

# Find duplicates based on 'SEGID'
duplicates = df_filtered.duplicated(subset=['SEGID'], keep=False)

# Filter the DataFrame to include only the duplicates
df_duplicates = df_filtered[duplicates]

df_duplicates = df_duplicates.sort_values(by=['SEGID','TDM_SUBAREAID'])

df_duplicates = df_duplicates.set_index(['SEGID'])

df_duplicates = df_duplicates.astype(int)

# This will give you a DataFrame with only the rows where 'SEGID' is duplicated
df_duplicates[df_duplicates['TDM_SUBAREAID']!=df_duplicates['SEG_SUBAREAID']]

,TDM_SUBAREAID,SEG_SUBAREAID
SEGID,,
0015_042.2,3,5
0065_002.4,0,1
0150_029.4,4,0
0150_030.0,4,0
0150_033.3,4,0
1822_000.0,0,1
2863_000.0,1,4
2863_000.0,4,1
2865_019.4,4,1


  - 0015_042.2: Remove SEGID from links in Dixie TDM (3)
 - 0065_002.4: Change SUBAREAID on links in USTM from 0 to 1
 - 0150_029.4, 015_030.0, 015_030.3 both segments where route leaves Summit County briefly into the corner of Duchesne County, so segments are not geographically located in SuWs assume should be in SuWs Model... so, seg summary should be updated... probably have to custom write exclusion into code, since SUBAREAID was calculated with spatial join
 - 1822_000.0: Majority of segment is in Utah County... Should split segment at county line, or change SUBAREAID on links in USTM from 0 to 1
 - 2865_019.4: Majority of segment is in Utah County... Should split segment at county line, or change SUBAREAID on links in USTM from 0 to 1


In [203]:
duplicateSegsInTdms

,SEGID,removeSUBAREAID
0,0015_042.2,3
1,0065_002.4,0
2,0150_029.4,0
3,0150_030.0,0
4,0150_033.3,0
5,1822_000.0,0
6,2863_000.0,4
7,2865_019.4,4


In [204]:
# Example dataframes
# df1 = pd.DataFrame({'SEGID': [1, 2], 'removeSUBAREAID': [101, 102]})
# dfMdlVolWithFac = pd.DataFrame({'SEGID': [1, 2, 3], 'SUBAREAID': [101, 102, 103], 'OtherColumn': ['A', 'B', 'C']})

# Assuming 'removeSUBAREAID' in df1 matches 'SUBAREAID' in dfMdlVolWithFac, and you want to remove records from dfMdlVolWithFac that have matching 'SEGID' and 'SUBAREAID'
# If 'removeSUBAREAID' is not meant to directly match 'SUBAREAID', adjust the merge accordingly.

# Merge the dataframes on 'SEGID' and 'SUBAREAID'/'removeSUBAREAID', indicating records to remove
merged_df = pd.merge(dfMdlVolWithFac, duplicateSegsInTdms, left_on=['SEGID', 'SUBAREAID'], right_on=['SEGID', 'removeSUBAREAID'], how='left', indicator=True)

# Filter out the records that have a match
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the columns added from df1 and the merge indicator to get back to the original dfMdlVolWithFac format
final_df = filtered_df.drop(columns=['removeSUBAREAID', '_merge'])

# final_df will be your dfMdlVolWithFac dataframe with the matched records removed
dfMdlVolWithFacDupRem = final_df
dfMdlVolWithFacDupRem

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122
1,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.031600,1.027600,0.984000,1.029600
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.031600,1.027600,0.984000,1.029600
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.031600,1.027600,0.984000,1.029600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.024300,1.010400,1.092400,1.017350
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.024300,1.010400,1.092400,1.017350
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.024300,1.010400,1.092400,1.017350
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.024300,1.010400,1.092400,1.017350


In [208]:
# CHECK TO SEE IF DUPLICATES STILL REMAIN

# Assuming dfMdlVol is your initial DataFrame
df_filtered2 = dfMdlVolWithFacDupRem[['SEGID','SUBAREAID']].drop_duplicates()

# Find duplicates based on 'SEGID'
duplicates2 = df_filtered2.duplicated(subset=['SEGID'], keep=False)

# Filter the DataFrame to include only the duplicates2
df_duplicates2 = df_filtered2[duplicates2]

# This will give you a DataFrame with only the rows where 'SEGID' is duplicated
df_duplicates2

,SEGID,SUBAREAID


# Calculate Model AADT

In [209]:
import numpy as np

# Your existing code for merging and filling NA
_df = dfMdlVolWithFacDupRem.copy()

# calculate AADT by subareaid
# divide by colWeekdayFactor
_df.loc[(~(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac)) & (_df[colFacWdAvg]> 0)), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg])
_df.loc[(~(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac)) & (_df[colFacWdAvg]==0)), 'modAadt'] = 0

# divide by average of Spring and Fall only for Summit/Wasatch Model
_df.loc[(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac) & (_df['FAC_SPRFAL']>0 )), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg] / _df['FAC_SPRFAL'])
_df.loc[(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac) & (_df['FAC_SPRFAL']==0)), 'modAadt'] = 0

# Convert to integer as you had before
_df['modAadt'] = _df['modAadt'].round(0).astype(int)

dfModAadt = _df

# Display the DataFrame
display(dfModAadt)

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122,0
1,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122,0
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.031600,1.027600,0.984000,1.029600,129
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.031600,1.027600,0.984000,1.029600,101
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.031600,1.027600,0.984000,1.029600,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50306,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.024300,1.010400,1.092400,1.017350,5852
50307,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.024300,1.010400,1.092400,1.017350,6720
50308,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.024300,1.010400,1.092400,1.017350,6379
50309,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.024300,1.010400,1.092400,1.017350,7806


# Get observed base year volumes

In [227]:
# GET BASE YEAR OBSERVED

# read intermediate CSV with base year observed AADT
_dfAadt = pd.read_csv('intermediate/aadt.csv')

# get SEGID and SUBAREAIDs from AADT dataframe  and merge to the desired source as indicated in dfObsVolSource
_dfSource = pd.merge(dfModAadt[['SEGID','SUBAREAID']].drop_duplicates(),dfSegsForObsVolSource,on=('SEGID','SUBAREAID'),how='left')

_df = pd.merge(_dfSource, _dfAadt, on=('SEGID','SOURCE'))

# filter by source and year
_df = _dfAadt[(_dfAadt['YEAR']<=baseYear)]

# get max row of max year for each segid... incase there is no observed 2019 data, but maybe a earlier year
# Then, group by 'SEGID' and find the index of the maximum 'YEAR' for each group
idx = _df.groupby('SEGID')['YEAR'].idxmax()

# Use the index to filter the rows
_df = _df.loc[idx]

_df = _df[['SEGID','AADT']]
_df.rename(columns={'AADT':'obsAadt'}, inplace=True)

dfObsAadt_BaseYear = _df

display(dfObsAadt_BaseYear)

,SEGID,obsAadt
133567,0006_000.0,415
133609,0006_000.7,415
133651,0006_016.0,415
133693,0006_046.0,372
133735,0006_060.2,372
...,...,...
132502,WFRC_8220,17096
132526,WFRC_8262,6672
132553,WFRC_8263,6672
132592,WFRC_8419,18324


In [228]:
dfObsAadt_BaseYear[dfObsAadt_BaseYear['SEGID']=='1695_000.0']

,SEGID,obsAadt
264925,1695_000.0,4122


In [229]:
# CALCULATE ADJUSTMENT FACTOR FROM BASE YEAR MODEL VS OBSERVED
dfModAadt_BaseYear = dfModAadt[dfModAadt['YEAR']==2019]

_df = pd.DataFrame.merge(dfModAadt_BaseYear[use_fields + ['modAadt']], dfObsAadt_BaseYear[['SEGID','obsAadt']], on='SEGID', how='outer')
_df.fillna(0,inplace=True)

_df['aadtAdjFac'] = _df['obsAadt'] - _df['modAadt']

_df.loc[(_df['obsAadt']==0), 'aadtAdjFac'] = 0

display(_df[_df['modAadt']==0])

dfModAdjFactor = _df[['SEGID','aadtAdjFac']]

display(dfModAdjFactor)


,SEGID,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,modAadt,obsAadt,aadtAdjFac
0,WFRC_8474,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,WFRC_8475,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
35,0006_239.5,0.0,0.0,0.0,0,0.0,0.0,12915.0,12915.0
42,0006_246.94,0.0,0.0,0.0,0,0.0,0.0,9034.0,9034.0
159,0015_368.1,0.0,0.0,0.0,0,0.0,0.0,28215.0,28215.0
...,...,...,...,...,...,...,...,...,...
8742,3183_001.4,0.0,0.0,0.0,0,0.0,0.0,500.0,500.0
8743,MAG_6021,0.0,0.0,0.0,0,0.0,0.0,1361.0,1361.0
8744,MAG_6140,0.0,0.0,0.0,0,0.0,0.0,500.0,500.0
8745,MAG_6254,0.0,0.0,0.0,0,0.0,0.0,1000.0,1000.0


,SEGID,aadtAdjFac
0,WFRC_8474,0.0
1,WFRC_8475,0.0
2,0006_000.0,286.0
3,0006_000.7,198.0
4,0006_016.0,337.0
...,...,...
8742,3183_001.4,500.0
8743,MAG_6021,1361.0
8744,MAG_6140,500.0
8745,MAG_6254,1000.0


In [213]:
# Function to round based on the given ranges
def custom_rounding(value):
    if 0 <= value < 100:
        return round(value / 10) * 10
    elif 100 <= value < 1000:
        return round(value / 50) * 50
    elif 1000 <= value < 10000:
        return round(value / 100) * 100
    elif 10000 <= value < 100000:
        return round(value / 500) * 500
    elif value >= 100000:
        return round(value / 1000) * 1000
    else:
        return value

In [234]:
# ADJUST ALL MODEL VOLUMES BASED ON BASE YEAR ADJ FACTOR

dfModWithAdj = pd.DataFrame.merge(dfModAadt, dfModAdjFactor, on='SEGID', how='left')
dfModWithAdj.fillna(0,inplace=True)
dfModWithAdj['aadtAdjFac'] = dfModWithAdj['aadtAdjFac'].astype(int)
dfModWithAdj['modForecast'] = (dfModWithAdj['modAadt'] + dfModWithAdj['aadtAdjFac']).apply(custom_rounding).astype(int)
dfModWithAdj

# remove 2019 since it is base
#dfModWithAdjNo2019 = dfModWithAdj[dfModWithAdj['YEAR']>2019]

# sort
#dfModWithAdjNo2019 = dfModWithAdjNo2019.sort_values(by=['SEGID', 'YEAR'], ascending=[True, True])

#dfModWithAdjNo2019

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122,0,0,0
1,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.007782,0.996462,1.058044,1.002122,0,0,0
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.031600,1.027600,0.984000,1.029600,129,286,400
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,99.7,2.0,Rural,1.0,2023.0,1.031600,1.027600,0.984000,1.029600,101,286,400
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,101.7,2.0,Rural,1.0,2028.0,1.031600,1.027600,0.984000,1.029600,103,286,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50264,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6392.6,3.0,Suburban,2.0,2023.0,1.024300,1.010400,1.092400,1.017350,5852,0,5900
50265,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,7340.4,3.0,Suburban,2.0,2028.0,1.024300,1.010400,1.092400,1.017350,6720,0,6700
50266,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,6968.0,3.0,Suburban,2.0,2032.0,1.024300,1.010400,1.092400,1.017350,6379,0,6400
50267,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,8527.4,3.0,Suburban,2.0,2042.0,1.024300,1.010400,1.092400,1.017350,7806,0,7800


In [236]:
dfModWithAdj[dfModWithAdj['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast
9432,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0827,1.2456,0.9243,1.16415,0,4122,4100


In [237]:
# check if base year adjusted equals base year observed

_df1 = dfModWithAdj[dfModWithAdj['YEAR']==baseYear]
_df2 = dfObsAadt_BaseYear

_df3 = pd.DataFrame.merge(_df1, _df2, on='SEGID')

_df3['ModVsObs'] = _df3['modForecast'] - _df3['obsAadt']

display(_df3[_df3['ModVsObs']!=0])


,SEGID,CO_FIPS,SAID_FAC,SOURCE,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast,obsAadt,ModVsObs
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,126.5,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960,129,286,400,415,-15
1,0006_000.7,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,213.6,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960,217,198,400,415,-15
2,0006_016.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,76.9,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960,78,337,400,415,-15
3,0006_046.0,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,76.9,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960,78,294,350,372,-22
4,0006_060.2,27.0,0.0,Segments_State_20231221_Draft.shp,0.0,80.3,2.0,Rural,1.0,2019.0,1.0316,1.0276,0.9840,1.02960,82,290,350,372,-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7260,WFRC_8010,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,2096.0,3.0,Transition,2.0,2019.0,1.0324,1.0107,1.0918,1.02155,1920,-808,1100,1112,-12
7261,WFRC_8220,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,12301.9,3.0,Suburban,3.0,2019.0,1.0243,1.0104,1.0924,1.01735,11261,5835,17000,17096,-96
7262,WFRC_8262,35.0,1.0,WF_Segments_20240326_Draft.shp,0.0,0.0,0.0,0,0.0,2019.0,1.0243,1.0104,1.0924,1.01735,0,6672,6700,6672,28
7263,WFRC_8263,35.0,1.0,WF_Segments_20240326_Draft.shp,1.0,4659.0,4.0,Suburban,2.0,2019.0,1.0243,1.0104,1.0924,1.01735,4265,2407,6700,6672,28


In [243]:
lstLockedSegments = gdfSegments[gdfSegments['F_AREA'].isin(lstLockForecastArea)]['SEGID'].drop_duplicates().to_list()
display(lstLockedSegments)

# merge locked with unlocked

if len(lstLockedSegments):

    print('Combine locked and unlocked')

    # export final file
    dfModWithAdj_Prev = pd.read_csv('intermediate/model-forecasts.csv')

    dfModWithAdj_Locked = dfModWithAdj_Prev[dfModWithAdj_Prev['SEGID'].isin(lstLockedSegments)].copy()

    dfModWithAdj_Unlocked = dfModWithAdj[~dfModWithAdj['SEGID'].isin(lstLockedSegments)].copy()

    # Concatenate the locked and unlocked DataFrames
    dfModWithAdjMerged = pd.concat([dfModWithAdj_Locked, dfModWithAdj_Unlocked])

    # Reset the index of the final DataFrame, if needed
    dfModWithAdjMerged.reset_index(drop=True, inplace=True)

else:
    dfModWithAdjMerged = dfModWithAdj

dfModWithAdjMerged

['0013_000.0',
 '0013_000.6',
 '0013_001.3',
 '0013_001.6',
 '0013_001.8',
 '0013_002.5',
 '0013_002.9',
 '0013_003.3',
 '0013_004.9',
 '0013_005.6',
 '0015_285.8',
 '0015_286.8',
 '0015_288.2',
 '0015_289.8',
 '0015_291.4',
 '0015_292.6',
 '0015_293.0',
 '0015_293.6',
 '0015_295.6',
 '0015_297.9',
 '0015_299.0',
 '0015_300.3',
 '0015_301.7',
 '0015_302.5',
 '0015_303.4',
 '0015_304.7',
 '0015_305.2',
 '0015_305.9',
 '0015_306.3',
 '0015_307.2',
 '0015_307.5',
 '0015_307.6',
 '0015_307.8',
 '0015_308.4',
 '0015_309.3',
 '0015_310.2',
 '0015_311.4',
 '0015_311.8',
 '0015_313.7',
 '0015_315.3',
 '0015_316.9',
 '0015_317.6',
 '0015_318.3',
 '0015_319.5',
 '0015_321.0',
 '0015_322.0',
 '0015_323.8',
 '0015_324.4',
 '0015_325.6',
 '0015_328.7',
 '0015_330.1',
 '0015_331.6',
 '0015_332.9',
 '0015_334.1',
 '0015_335.8',
 '0015_337.0',
 '0015_338.5',
 '0015_339.1',
 '0015_340.3',
 '0015_342.0',
 '0015_343.1',
 '0015_343.9',
 '0015_345.0',
 '0015_346.7',
 '0015_349.4',
 '0015_351.9',
 '0015_357

Combine locked and unlocked


,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SOURCE,SAID_FAC,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast,CO_FIPS,SUBAREAID
0,0013_000.0,13747.6,2.0,Suburban,4.0,2019.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,12585.0,6929.0,19500.0,NaN,NaN
1,0013_000.0,14506.3,2.0,Suburban,4.0,2023.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,13279.0,6929.0,20000.0,NaN,NaN
2,0013_000.0,15215.0,2.0,Suburban,4.0,2028.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,13928.0,6929.0,21000.0,NaN,NaN
3,0013_000.0,16248.9,2.0,Suburban,4.0,2032.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,14874.0,6929.0,22000.0,NaN,NaN
4,0013_000.0,15734.2,2.0,Suburban,4.0,2042.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,14403.0,6929.0,21500.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50320,WFRC_8119,6917.4,3.0,Suburban,2.0,2028.0,WF_Segments_20240326_Draft.shp,1.0,1.0324,1.0107,1.0918,1.02155,6336.0,0.0,6300.0,35.0,1.0
50321,WFRC_8119,7390.1,3.0,Suburban,2.0,2032.0,WF_Segments_20240326_Draft.shp,1.0,1.0324,1.0107,1.0918,1.02155,6769.0,0.0,6800.0,35.0,1.0
50322,WFRC_8119,11818.8,3.0,Suburban,2.0,2042.0,WF_Segments_20240326_Draft.shp,1.0,1.0324,1.0107,1.0918,1.02155,10825.0,0.0,11000.0,35.0,1.0
50323,WFRC_8119,12044.4,3.0,Urban,2.0,2050.0,WF_Segments_20240326_Draft.shp,1.0,1.0324,1.0107,1.0918,1.02155,11032.0,0.0,11000.0,35.0,1.0


In [244]:
# export final filedfModWithAdjMerged
dfModWithAdjMerged.to_csv('intermediate/model-forecasts.csv', index=False)